In [1]:
import json
import subprocess
from pdf2image import convert_from_path
import os

In [2]:
def extract_metrics(json_path):
    with open(json_path, 'r') as file:
        data = json.load(file)
    
    model_name = data.get("model_name", "Unknown Model")
    accuracy = data.get("accuracy", data['cf_report'].get('accuracy'))
    f1_weighted = data["cf_report"]["weighted avg"].get("f1-score", 0)
    f1_macro = data["cf_report"]["macro avg"].get("f1-score", 0)
    try:
        f1_micro = data["cf_report"]["micro avg"].get("f1-score", 0)
    except KeyError:
        f1_micro = 0
    metrics = {
        "Model": model_name,
        "Acurácia (%)": round(accuracy * 100, 2),
        "F1-weighted (%)": round(f1_weighted * 100, 2),
        "F1-macro (%)": round(f1_macro * 100, 2),
        "F1-micro (%)": round(f1_micro * 100, 2)
        
    }
    return metrics

In [3]:
def generate_table(json_paths):
    all_metrics = []
    include_f1_micro = False
    
    for path in json_paths:
        metrics = extract_metrics(path)
        all_metrics.append(metrics)
        # Verifica se há pelo menos uma ocorrência de F1-micro maior que 0
        if metrics["F1-micro (%)"] > 0:
            include_f1_micro = True
    
    # Encontrar os valores máximos de cada métrica
    max_f1_weighted = max(m["F1-weighted (%)"] for m in all_metrics)
    max_f1_macro = max(m["F1-macro (%)"] for m in all_metrics)
    max_accuracy = max(m["Acurácia (%)"] for m in all_metrics)
    max_f1_micro = max(m["F1-micro (%)"] for m in all_metrics) if include_f1_micro else None
    
    table_latex = r"""\begin{table}[ht]
\centering
\renewcommand{\arraystretch}{1.2}
\caption{Desempenho dos modelos para a tarefa de classificação.}
\label{tab:tableDesempenhoModelos}
\vspace{4pt} % Espaço vertical adicionado antes da legenda
\begin{tabular}{l"""

    table_latex += " c"  
    if include_f1_micro:
        table_latex += " c"  
    table_latex += " c"  
    table_latex += " c"  
    
    table_latex += "}\n\\toprule\n\\textbf{Modelo} & \\textbf{Acurácia (\\%)}"
    
    if include_f1_micro:
        table_latex += " & \\textbf{F1-micro (\\%)}"
    
    table_latex += " & \\textbf{F1-macro (\\%)} & \\textbf{F1-weighted (\\%)} \\\\\n\\midrule\n"
    
    for metrics in all_metrics:
        table_latex += f'{metrics["Model"]}'
        
        accuracy_str = f"\\textbf{{{metrics['Acurácia (%)']}}}" if metrics["Acurácia (%)"] == max_accuracy else f"{metrics['Acurácia (%)']}"
        table_latex += f" & {accuracy_str}"
        
        if include_f1_micro:
            f1_micro_str = f"\\textbf{{{metrics['F1-micro (%)']}}}" if metrics["F1-micro (%)"] == max_f1_micro else f"{metrics['F1-micro (%)']}"
            table_latex += f" & {f1_micro_str}"
        
        f1_macro_str = f"\\textbf{{{metrics['F1-macro (%)']}}}" if metrics["F1-macro (%)"] == max_f1_macro else f"{metrics['F1-macro (%)']}"
        table_latex += f" & {f1_macro_str}"
        
        f1_weighted_str = f"\\textbf{{{metrics['F1-weighted (%)']}}}" if metrics["F1-weighted (%)"] == max_f1_weighted else f"{metrics['F1-weighted (%)']}"
        table_latex += f" & {f1_weighted_str}"
        
        table_latex += " \\\\\n"
    
    table_latex += r"""\bottomrule
\end{tabular}
\end{table}
"""
    
    return table_latex

In [5]:
dir_metricas = "/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Metricas/Secoes/CrossValidation/NovasMetricas"

In [6]:
arquivos_json = []
for arquivo in os.listdir(dir_metricas):
    if arquivo.endswith(".json"):
        arquivos_json.append(os.path.join(dir_metricas, arquivo))    

In [8]:
latex_table = generate_table(arquivos_json)
with open("table.tex", "w") as file:
    file.write(latex_table)